<a href="https://colab.research.google.com/github/asikariaf/notebooks/blob/main/FirstModel_LeNet5_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Use correct device
device = ( "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

In [3]:
# Define some constants
batch_size = 64
num_classes = 10         # number of outputs of neural network. i.e., it is classifying into one of 10 classes
learning_rate = 0.001
num_epochs = 10

In [4]:
training_data = torchvision.datasets.MNIST(root="mnist", train=True, download=True, transform=ToTensor())
test_data = torchvision.datasets.MNIST(root="mnist", train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

100%|██████████| 9912422/9912422 [00:00<00:00, 320256595.63it/s]

Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 113105222.99it/s]


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 175170117.19it/s]

Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 20911667.14it/s]

Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw



In [ ]:
print(training_data[200][0].shape)
print(len(training_data))
print(len(train_dataloader))


In [6]:
#Model definition is from: https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/

#Define the network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(256, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [7]:
# Instantiate Model
model = LeNet5(num_classes).to(device)

#Setting the loss function
loss_fn = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, batch_data in enumerate(dataloader):
        images, labels = batch_data
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        predictions = model(images)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            loss = loss.item()

        if batch_num > 0 and (batch_num+1) % 100 == 0:
            current = (batch_num + 1) * batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

In [9]:
def test_one_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            prediction = model(images)
            loss += loss_fn(prediction, labels).item()
            correct += (prediction.argmax(1) == labels).type(torch.float).sum().item()

    loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    return loss

In [ ]:
for t in range(num_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_one_epoch(train_dataloader, model, loss_fn, optimizer)
    test_one_epoch(test_dataloader, model, loss_fn)
print("Done!")